# Running BOA Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib

import yaml
import shutil

import boa
from wrapper import Wrapper

In [2]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("boa_runs*")
for path in old_runs:
    shutil.rmtree(path)

## Loading the Config File

In [3]:
config_path = pathlib.Path().resolve() / "single_config.yaml"

Here we can see what the configuration file looks like

In [4]:
with open(config_path, 'r') as f:
    file_contents = f.read()
    print (file_contents)

# Single objective optimization config
optimization_options:
    objective_options:
        objectives:
            - name: Cosine8
    trials: 50
    append_timestamp: False
parameters:
    x0:
        type: range
        bounds: [0.0, 1.0]
    x1:
        type: range
        bounds: [0.0, 1.0]
    x2:
        type: range
        bounds: [0.0, 1.0]
    x3:
        type: range
        bounds: [0.0, 1.0]
    x4:
        type: range
        bounds: [0.0, 1.0]
    x5:
        type: range
        bounds: [0.0, 1.0]
    x6:
        type: range
        bounds: [0.0, 1.0]
    x7:
        type: range
        bounds: [0.0, 1.0]
# These are all defaults, so we don't need to specify them in this case
#script_options:
#    wrapper_path: ./wrapper.py
#    wrapper_name: Wrapper
#    working_dir: .
#    experiment_dir: ... # this is where boa will write logs to by default
                         # if not specified it will be working_dir/experiment_name
#    append_timestamp: True
# This last option 

we need the config normalized, which modifies the parameter section
into a less user friendly form, but what the downstream libraries need

In [5]:
config = boa.load_jsonlike(config_path)

## Define Our Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [6]:
Wrapper.path()

PosixPath('/Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/wrapper.py')

In [7]:
with open(Wrapper.path(), 'r') as f:
    file_contents = f.read()
    print (file_contents)

import numpy as np
from ax.utils.measurement.synthetic_functions import from_botorch
from botorch.test_functions.synthetic import Cosine8

import boa

cosine8 = from_botorch(Cosine8())


def black_box_model(X) -> float:
    result = -cosine8(X)
    return result


class Wrapper(boa.BaseWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = {}

    def run_model(self, trial) -> None:
        X = np.array([parameter for parameter in trial.arm.parameters.values()])
        # This is a silly toy function, in reality,
        # you could instead import your model main() function and use that, and then collect the results.
        # You could also call an external script to start a model run from Bash or elsewhere.
        self.data[trial.index] = black_box_model(X)

    def set_trial_status(self, trial) -> None:
        data_exists = self.data.get(trial.index)
        if data_exists:
            trial.mark_completed()

    def fetch_

## Initialize our Setup

In [8]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-10 11:37:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x0. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-10 11:37:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-10 11:37:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-10 11:37:38] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 07-10 

(Scheduler(experiment=Experiment(boa_runs), generation_strategy=GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 16 trials, GPEI for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <wrapper.Wrapper at 0x158872d40>)

## Run our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [9]:
scheduler = controller.run()

[INFO 2023-07-10 11:37:38,278 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/boa_runs_20230710T113738
Start Time: 20230710T113738
Version: 0.8.3.dev1+g0636f51.d20230411

##############################################

[INFO 07-10 11:37:38] Scheduler: Running trials [0]...
[INFO 07-10 11:37:39] Scheduler: Running trials [1]...
[INFO 07-10 11:37:40] Scheduler: Running trials [2]...
[INFO 07-10 11:37:41] Scheduler: Running trials [3]...
[INFO 07-10 11:37:41] Scheduler: Running trials [4]...
[INFO 07-10 11:37:42] Scheduler: Running trials [5]...
[INFO 07-10 11:37:43] Scheduler: Running trials [6]...
[INFO 07-10 11:37:44] Scheduler: Running trials [7]...
[INFO 07-10 11:37:45] Scheduler: Running trials [8]...
[INFO 07-10 11:37:46] Scheduler: Running trials [9]...
[INFO 07-10 11:37:47] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 07-10 11:37:47] Scheduler: F

## Get the Best Trial and Output All Trials

`best_fitted_trials` uses the data to do a fitting from all trials and with the noise levels you provided (or if no noise levels was provided, it assumed an unknown level of noise and inferred the noise level from the trial runs)

In [10]:
trial = scheduler.best_fitted_trials()
trial

/Users/madelinescyphers/miniconda3/envs/boa-dev/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)
/Users/madelinescyphers/miniconda3/envs/boa-dev/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


{49: {'params': {'x0': 0.35172402518319096,
   'x1': 0.3698520404111157,
   'x2': 0.3666874377748964,
   'x3': 0.3891053518012792,
   'x4': 0.3734271631500215,
   'x5': 0.0,
   'x6': 0.36002879332283894,
   'x7': 0.35626455543974167},
  'means': {'Cosine8': 0.2462324826494231},
  'cov_matrix': {'Cosine8': {'Cosine8': 1.3891737938407046e-06}}}}

if you need the exact points of the best trial, maybe because you need the trial number of the best trial to plot results, or for any other reason, `best_raw_trails` does not do any fitting

In [11]:
trial = scheduler.best_raw_trials()
trial

{46: {'params': {'x0': 0.35172402518319096,
   'x1': 0.3698520404111157,
   'x2': 0.3666874377748964,
   'x3': 0.3891053518012792,
   'x4': 0.3734271631500215,
   'x5': 0.0,
   'x6': 0.36002879332283894,
   'x7': 0.35626455543974167},
  'means': {'Cosine8': 0.24594534280361183},
  'cov_matrix': {'Cosine8': {'Cosine8': 0.0}}}}

## Output a DataFrame of All Trials

In [12]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,Cosine8,x0,x1,x2,x3,x4,x5,x6,x7
0,0,0_0,COMPLETED,Sobol,2.029324,0.990547,0.288610,0.346135,0.301916,0.392528,0.110616,0.251962,0.694829
1,1,1_0,COMPLETED,Sobol,1.646673,0.262968,0.633469,0.616036,0.388401,0.427373,0.075654,0.736025,0.068805
2,2,2_0,COMPLETED,Sobol,2.144039,0.449725,0.480685,0.448159,0.548387,0.476969,0.054146,0.188638,0.978092
3,3,3_0,COMPLETED,Sobol,2.753313,0.431875,0.352470,0.052401,0.387660,0.960082,0.576974,0.864202,0.656100
4,4,4_0,COMPLETED,Sobol,3.049808,0.417872,0.647828,0.534399,0.174281,0.122930,0.771529,0.943236,0.778014
5,5,5_0,COMPLETED,Sobol,3.438582,0.895702,0.123685,0.017906,0.944492,0.357977,0.397257,0.967300,0.830473
6,6,6_0,COMPLETED,Sobol,2.069711,0.730564,0.433245,0.129033,0.007862,0.276506,0.861777,0.722698,0.493578
7,7,7_0,COMPLETED,Sobol,3.978929,0.119162,0.841748,0.990021,0.173180,0.671806,0.980801,0.351636,0.768843
8,8,8_0,COMPLETED,Sobol,2.448196,0.872235,0.815817,0.038919,0.380534,0.389911,0.711213,0.233549,0.776188
9,9,9_0,COMPLETED,Sobol,2.355544,0.286572,0.689979,0.266366,0.717407,0.132874,0.584265,0.825217,0.396867
